In [1]:
import cobra
import pandas as pd
from cobra import Model, Reaction, Metabolite

import matplotlib.pyplot as plt
import numpy as np

import os
import copy
import time

from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

from libsbml import *
reader = SBMLReader()

In [2]:
new_model = cobra.io.read_sbml_model('Models/Recon2_Influenza_consistent.xml')
print(len(new_model.reactions))
print(new_model.optimize().objective_value)


5227
0.2925755033557048


In [3]:
def analyse_specific_model(directory, filename): 
    from cobra import Model, Reaction, Metabolite
    '''
    filename = e.g. lung_15.csv !!with ending!!
    '''
    
    #Reading in the files from matlab
    try:
        
        tissue_reacts = pd.read_csv(directory + filename, header=None)
        
        list_of_tissue_reacts = list(tissue_reacts[0])
        list_of_tissue_reacts = [x - 1 for x in list_of_tissue_reacts]
        
        new_tissue_model = Model('Consistent_Recon2')

        for i in list_of_tissue_reacts:
            new_tissue_model.add_reactions([new_model.reactions[i]])

        NumReacts = len(new_tissue_model.reactions)
        
        new_tissue_model.objective = 'biomass_reaction'
        BOF = new_tissue_model.optimize().objective_value
        
        #reactions
        #react_ko = single_reaction_deletion(new_tissue_model)
        #genes
        react_ko = single_gene_deletion(new_tissue_model)
        react_ko = react_ko.rename(columns={'growth': 'growth_host', 'status': 'status_host'})

            
        with new_tissue_model:
            new_tissue_model.objective='VBOF'
            VBOF = new_tissue_model.optimize().objective_value
            
            #reactions
            #react_ko_vbof = single_reaction_deletion(new_tissue_model)
            #genes
            react_ko_vbof = single_gene_deletion(new_tissue_model)
            react_ko_vbof = react_ko_vbof.rename(columns={'growth': 'growth_virus', 'status': 'status_virus'})
            
        #Analysis-result
        merged = react_ko.merge(react_ko_vbof, left_on='ids', right_on='ids')
        
        new_indices = [list(i)[0] for i in merged.index]
        merged = merged.set_index([pd.Index(new_indices)])
        
        percent_bof = [merged.loc[i]['growth_host']/BOF*100 for i in merged.index]
        merged.insert(loc = 0, column='%growth_host', value=percent_bof)
        
        percent_vbof = [merged.loc[i]['growth_virus']/VBOF*100 for i in merged.index]
        merged.insert(loc = 0, column='%growth_virus', value=percent_vbof)
        
        possible_targets = []
        relevant = []
        
        for i in merged.index:
            #if merged.loc[i]['%growth_virus'] < 50 and merged.loc[i]['%growth_virus'] < merged.loc[i]['%growth_host']:
            if merged.loc[i]['%growth_virus'] < 50 and merged.loc[i]['%growth_host'] > 80 and i != 'VBOF':
                possible_targets.append([i, merged.loc[i]['%growth_virus'], merged.loc[i]['%growth_host']])
                #print(i, merged.loc[i]['%growth_virus'], merged.loc[i]['%growth_host'])
            if merged.loc[i]['%growth_virus'] <= 0.05:
                relevant.append(i)
                
        if len(possible_targets) == 0:
            possible_targets = 0
        if len(relevant) == 0:
            relevant = 0
            
    except:
        BOF = np.nan
        VBOF = np.nan
        NumReacts = np.nan
        possible_targets = np.nan
        relevant = np.nan

    
    return([BOF, VBOF, NumReacts, possible_targets, relevant])


In [4]:
#testing
directory = 'CoreModelsInfluenza/'
start_time = time.time()
bof, vbof, reacts, targets, relevant = analyse_specific_model(directory, 'GSM5740432_S2_AAACCCACAATCAAGA-1.csv')#('A10_B000232_B008361_S10_mm10_plus_7_0')#('K22_MAA000891_3_10_M_1_1_1_1')
print('Time: ', time.time() - start_time)
print(bof, vbof, reacts, targets, relevant)

Time:  2.97479510307312
0.004483041295214671 0.01892189288013699 1574 0 ['HGNC:758', 'HGNC:2292', 'HGNC:2279', 'HGNC:24380', 'HGNC:12582', 'HGNC:16232', 'HGNC:2289', 'HGNC:30863', 'HGNC:2269', 'HGNC:30862', 'HGNC:2579', 'HGNC:2285', 'HGNC:10682', 'HGNC:4433', 'HGNC:18170', 'HGNC:24381', 'HGNC:4693', 'HGNC:2291', 'HGNC:12590', 'HGNC:11067', 'HGNC:2265', 'HGNC:12585', 'HGNC:7892', 'HGNC:2267', 'HGNC:11063', 'HGNC:3700', 'HGNC:10683', 'HGNC:29594', 'HGNC:2288', 'HGNC:12587', 'HGNC:2287', 'HGNC:2280', 'HGNC:16484', 'HGNC:7419', 'HGNC:24382', 'HGNC:2277', 'HGNC:11064', 'HGNC:746', 'HGNC:12586', 'HGNC:7427', 'HGNC:9020', 'HGNC:2294', 'HGNC:7421', 'HGNC:10971', 'HGNC:12563', 'HGNC:10680', 'HGNC:2501', 'HGNC:10681', 'HGNC:7422']


In [5]:
### All models
directory = 'CoreModelsInfluenza/'
bof_dict = {}
vbof_dict = {}
react_dict = {}
target_dict = {}
relevant_dict = {}
counter = 0

for filename in os.listdir(directory): #infected_epithelial: 
    if counter %200 == 0:
        print(counter)
    bof, vbof, reacts, target, relevant = analyse_specific_model(directory, filename)
    bof_dict[filename] = bof
    vbof_dict[filename] = vbof
    react_dict[filename]= reacts
    target_dict[filename] = target
    relevant_dict[filename] = relevant
    counter += 1
    

0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400


In [6]:
new_dict = {}
for key in bof_dict.keys():
    if key != '.DS_Store':
        new_dict[key] = [bof_dict[key], vbof_dict[key], react_dict[key], target_dict[key], relevant_dict[key]]
evaluation = pd.DataFrame.from_dict(new_dict).T
evaluation = evaluation.rename(columns={0: "BOF", 1: "VBOF", 2: "Number_Reactions", 3: 'Targets (Gene,%VBOF, %BOF)', 
                                                   4: 'Virus relevant'})
print(len(evaluation))
evaluation = evaluation.dropna()
print(len(evaluation))
evaluation.head()

3503
3066


,BOF,VBOF,Number_Reactions,"Targets (Gene,%VBOF, %BOF)",Virus relevant
GSM5740432_S2_GGGTTTACACAAGTTC-1.csv,0.00654094,0.0858842,1818,"[[HGNC:8016, 35.890905400818994, 100.0], [HGNC...","[HGNC:10979, HGNC:4175, HGNC:18170, HGNC:8091,..."
GSM5740432_S2_ACGTTCCAGCTGAGCA-1.csv,0.00358027,0.00946095,1518,0,"[HGNC:10452, HGNC:12586, HGNC:7427, HGNC:2277,..."
GSM5740432_S2_ACTACGAAGATGACCG-1.csv,0.00448304,0.0189219,1711,0,"[HGNC:2265, HGNC:24380, HGNC:2289, HGNC:2269, ..."
GSM5740432_S2_AGTACCAAGGGCGAGA-1.csv,0.00448304,0.0103348,817,"[[HGNC:2995, 35.55704366170918, 100.0], [HGNC:...","[HGNC:4693, HGNC:2291, HGNC:4378, HGNC:848, HG..."
GSM5740432_S2_ACTTCCGAGTATCTGC-1.csv,0.00448304,0.0189219,1124,0,"[HGNC:8907, HGNC:10293, HGNC:746, HGNC:2205, H..."


In [7]:
evaluation.to_csv('csvs/Influenza_evaluation.csv')

